In [1]:
from ghostnet.ghost_net import ghost_net
from keras.datasets import mnist
import pandas as pd
import numpy as np
import torch
from torch.nn.utils.rnn import pack_padded_sequence
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn,optim
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt

# 检验GPU是否可用
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
import  tensorflow as  tf

#加载cifar-10数据集
def dataLoad():
    (train_data, train_label), (test_data, test_label) =  tf.keras.datasets.cifar10.load_data()
    #数据集进行归一化
    train_data=train_data/255
    test_data=test_data/255
    #将标签数据集从数组类型array修改成整形类型int
    train_label.astype(np.int)
    test_label.astype(np.int)
    return (train_data, train_label), (test_data, test_label)

In [3]:
(train_data, train_label), (test_data, test_label) = dataLoad()

In [4]:
train_data = train_data.reshape((50000,3,32,32))
test_data = test_data.reshape((-1,3,32,32))
train_data = torch.FloatTensor(train_data)
test_data = torch.FloatTensor(test_data)
train_label = torch.LongTensor(train_label)
test_label = torch.LongTensor(test_label)

In [5]:
class MyDataset(Dataset):
    
    # 初始化
    def __init__(self, pics, lable):
        # 读入数据
        self.data = pics
        self.lable = lable
        
    # 返回df的长度
    def __len__(self):
        return len(self.data)
    
    # 获取第idx+1列的数据
    def __getitem__(self, idx):
        return self.data[idx],self.lable[idx]

In [6]:
my_dataset_train = MyDataset(train_data, train_label)
my_dataset_test = MyDataset(test_data, test_label)
train_loader = DataLoader(dataset = my_dataset_train, batch_size = 64, shuffle = True)
test_loader = DataLoader(dataset = my_dataset_test, batch_size = 64, shuffle = True)

In [8]:
model = ghost_net(width_mult=1.0,num_classes=10).to(device)
# model = Net().to(device)
optimizer = optim.Adam(model.parameters(),lr=0.001)  # 优化方式为mini-batch momentum-SGD，并采用L2正则化（权重衰减）
criterion = nn.CrossEntropyLoss()

In [10]:
# model.train()
batch_idx = 1
epoches = 200
for epoch in range(epoches):
    batch_idx = 1
    model.train()
    for line in train_loader:
        data = line[0].to(device)
        label = line[1].to(device)
        output = model(data)
    #     print(output.size())
    #     print(label.size())
        label = label.squeeze(1)
        output = torch.log_softmax(output,dim = 1)
        loss = criterion(output, label)
        optimizer.zero_grad() #这一步可太TM重要了
        loss.backward()
        optimizer.step()
#         print(loss)
        if(batch_idx)%100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        batch_idx += 1
        
    correct = 0
    model.eval()
    for line in test_loader:
        data = line[0].to(device)
        label = line[1].to(device)
        output = model(data)
        output = torch.log_softmax(output,dim = 1)
        pred = output.max(1, keepdim=True)[1]# 找到概率最大的下标
        correct += pred.eq(label.view_as(pred)).sum().item()
    print("测试集上准确率为:",(correct/10000)*100,"%")

Train Epoch: 0 [6400/50000 (13%)]	Loss: 0.389953
Train Epoch: 0 [12800/50000 (26%)]	Loss: 0.360821
Train Epoch: 0 [19200/50000 (38%)]	Loss: 0.478962
Train Epoch: 0 [25600/50000 (51%)]	Loss: 0.360452
Train Epoch: 0 [32000/50000 (64%)]	Loss: 0.331908
Train Epoch: 0 [38400/50000 (77%)]	Loss: 0.285842
Train Epoch: 0 [44800/50000 (90%)]	Loss: 0.497511
测试集上准确率为: 51.67 %
Train Epoch: 1 [6400/50000 (13%)]	Loss: 0.203075
Train Epoch: 1 [12800/50000 (26%)]	Loss: 0.401362
Train Epoch: 1 [19200/50000 (38%)]	Loss: 0.433753
Train Epoch: 1 [25600/50000 (51%)]	Loss: 0.612859
Train Epoch: 1 [32000/50000 (64%)]	Loss: 0.169235
Train Epoch: 1 [38400/50000 (77%)]	Loss: 0.309953
Train Epoch: 1 [44800/50000 (90%)]	Loss: 0.381254
测试集上准确率为: 49.53 %
Train Epoch: 2 [6400/50000 (13%)]	Loss: 0.320807
Train Epoch: 2 [12800/50000 (26%)]	Loss: 0.477955
Train Epoch: 2 [19200/50000 (38%)]	Loss: 0.234361
Train Epoch: 2 [25600/50000 (51%)]	Loss: 0.217578
Train Epoch: 2 [32000/50000 (64%)]	Loss: 0.200207
Train Epoch: 2 [3

Train Epoch: 21 [44800/50000 (90%)]	Loss: 0.196799
测试集上准确率为: 52.239999999999995 %
Train Epoch: 22 [6400/50000 (13%)]	Loss: 0.088279
Train Epoch: 22 [12800/50000 (26%)]	Loss: 0.082912
Train Epoch: 22 [19200/50000 (38%)]	Loss: 0.112760
Train Epoch: 22 [25600/50000 (51%)]	Loss: 0.060438
Train Epoch: 22 [32000/50000 (64%)]	Loss: 0.123945
Train Epoch: 22 [38400/50000 (77%)]	Loss: 0.128004
Train Epoch: 22 [44800/50000 (90%)]	Loss: 0.043474
测试集上准确率为: 52.980000000000004 %
Train Epoch: 23 [6400/50000 (13%)]	Loss: 0.090296
Train Epoch: 23 [12800/50000 (26%)]	Loss: 0.074126
Train Epoch: 23 [19200/50000 (38%)]	Loss: 0.131703
Train Epoch: 23 [25600/50000 (51%)]	Loss: 0.224776
Train Epoch: 23 [32000/50000 (64%)]	Loss: 0.146342
Train Epoch: 23 [38400/50000 (77%)]	Loss: 0.114328
Train Epoch: 23 [44800/50000 (90%)]	Loss: 0.121590
测试集上准确率为: 52.39 %
Train Epoch: 24 [6400/50000 (13%)]	Loss: 0.147199
Train Epoch: 24 [12800/50000 (26%)]	Loss: 0.126183
Train Epoch: 24 [19200/50000 (38%)]	Loss: 0.097350
Train

Train Epoch: 43 [38400/50000 (77%)]	Loss: 0.208481
Train Epoch: 43 [44800/50000 (90%)]	Loss: 0.075290
测试集上准确率为: 52.94 %
Train Epoch: 44 [6400/50000 (13%)]	Loss: 0.199863
Train Epoch: 44 [12800/50000 (26%)]	Loss: 0.108221
Train Epoch: 44 [19200/50000 (38%)]	Loss: 0.128969
Train Epoch: 44 [25600/50000 (51%)]	Loss: 0.199372
Train Epoch: 44 [32000/50000 (64%)]	Loss: 0.417784
Train Epoch: 44 [38400/50000 (77%)]	Loss: 0.239348
Train Epoch: 44 [44800/50000 (90%)]	Loss: 0.214572
测试集上准确率为: 51.92 %
Train Epoch: 45 [6400/50000 (13%)]	Loss: 0.275469
Train Epoch: 45 [12800/50000 (26%)]	Loss: 0.261210
Train Epoch: 45 [19200/50000 (38%)]	Loss: 0.284263
Train Epoch: 45 [25600/50000 (51%)]	Loss: 0.268791
Train Epoch: 45 [32000/50000 (64%)]	Loss: 0.135004
Train Epoch: 45 [38400/50000 (77%)]	Loss: 0.241707
Train Epoch: 45 [44800/50000 (90%)]	Loss: 0.201801
测试集上准确率为: 52.5 %
Train Epoch: 46 [6400/50000 (13%)]	Loss: 0.229943
Train Epoch: 46 [12800/50000 (26%)]	Loss: 0.128067
Train Epoch: 46 [19200/50000 (38

Train Epoch: 65 [19200/50000 (38%)]	Loss: 0.207548
Train Epoch: 65 [25600/50000 (51%)]	Loss: 0.168362
Train Epoch: 65 [32000/50000 (64%)]	Loss: 0.133982
Train Epoch: 65 [38400/50000 (77%)]	Loss: 0.097614
Train Epoch: 65 [44800/50000 (90%)]	Loss: 0.203144
测试集上准确率为: 52.559999999999995 %
Train Epoch: 66 [6400/50000 (13%)]	Loss: 0.147585
Train Epoch: 66 [12800/50000 (26%)]	Loss: 0.105261
Train Epoch: 66 [19200/50000 (38%)]	Loss: 0.022112
Train Epoch: 66 [25600/50000 (51%)]	Loss: 0.113107
Train Epoch: 66 [32000/50000 (64%)]	Loss: 0.083881
Train Epoch: 66 [38400/50000 (77%)]	Loss: 0.089322
Train Epoch: 66 [44800/50000 (90%)]	Loss: 0.114666
测试集上准确率为: 53.04 %
Train Epoch: 67 [6400/50000 (13%)]	Loss: 0.090378
Train Epoch: 67 [12800/50000 (26%)]	Loss: 0.176742
Train Epoch: 67 [19200/50000 (38%)]	Loss: 0.090790
Train Epoch: 67 [25600/50000 (51%)]	Loss: 0.077166
Train Epoch: 67 [32000/50000 (64%)]	Loss: 0.162759
Train Epoch: 67 [38400/50000 (77%)]	Loss: 0.051803
Train Epoch: 67 [44800/50000 (90%)]

测试集上准确率为: 53.31 %
Train Epoch: 87 [6400/50000 (13%)]	Loss: 0.137280
Train Epoch: 87 [12800/50000 (26%)]	Loss: 0.025306
Train Epoch: 87 [19200/50000 (38%)]	Loss: 0.032243
Train Epoch: 87 [25600/50000 (51%)]	Loss: 0.071221
Train Epoch: 87 [32000/50000 (64%)]	Loss: 0.091068
Train Epoch: 87 [38400/50000 (77%)]	Loss: 0.037442
Train Epoch: 87 [44800/50000 (90%)]	Loss: 0.097085
测试集上准确率为: 53.1 %
Train Epoch: 88 [6400/50000 (13%)]	Loss: 0.272546
Train Epoch: 88 [12800/50000 (26%)]	Loss: 0.024927
Train Epoch: 88 [19200/50000 (38%)]	Loss: 0.089533
Train Epoch: 88 [25600/50000 (51%)]	Loss: 0.050374
Train Epoch: 88 [32000/50000 (64%)]	Loss: 0.068335
Train Epoch: 88 [38400/50000 (77%)]	Loss: 0.137306
Train Epoch: 88 [44800/50000 (90%)]	Loss: 0.030440
测试集上准确率为: 51.76 %
Train Epoch: 89 [6400/50000 (13%)]	Loss: 0.034455
Train Epoch: 89 [12800/50000 (26%)]	Loss: 0.037366
Train Epoch: 89 [19200/50000 (38%)]	Loss: 0.095582
Train Epoch: 89 [25600/50000 (51%)]	Loss: 0.238306
Train Epoch: 89 [32000/50000 (64

Train Epoch: 108 [32000/50000 (64%)]	Loss: 0.082728
Train Epoch: 108 [38400/50000 (77%)]	Loss: 0.080217
Train Epoch: 108 [44800/50000 (90%)]	Loss: 0.109088
测试集上准确率为: 53.879999999999995 %
Train Epoch: 109 [6400/50000 (13%)]	Loss: 0.035484
Train Epoch: 109 [12800/50000 (26%)]	Loss: 0.023632
Train Epoch: 109 [19200/50000 (38%)]	Loss: 0.027860
Train Epoch: 109 [25600/50000 (51%)]	Loss: 0.107283
Train Epoch: 109 [32000/50000 (64%)]	Loss: 0.009893
Train Epoch: 109 [38400/50000 (77%)]	Loss: 0.148061
Train Epoch: 109 [44800/50000 (90%)]	Loss: 0.061490
测试集上准确率为: 53.42 %
Train Epoch: 110 [6400/50000 (13%)]	Loss: 0.053767
Train Epoch: 110 [12800/50000 (26%)]	Loss: 0.337753
Train Epoch: 110 [19200/50000 (38%)]	Loss: 0.256832
Train Epoch: 110 [25600/50000 (51%)]	Loss: 0.564516
Train Epoch: 110 [32000/50000 (64%)]	Loss: 0.539325
Train Epoch: 110 [38400/50000 (77%)]	Loss: 0.384526
Train Epoch: 110 [44800/50000 (90%)]	Loss: 0.268524
测试集上准确率为: 52.42 %
Train Epoch: 111 [6400/50000 (13%)]	Loss: 0.135537


Train Epoch: 129 [38400/50000 (77%)]	Loss: 0.100506
Train Epoch: 129 [44800/50000 (90%)]	Loss: 0.038113
测试集上准确率为: 53.459999999999994 %
Train Epoch: 130 [6400/50000 (13%)]	Loss: 0.057312
Train Epoch: 130 [12800/50000 (26%)]	Loss: 0.229376
Train Epoch: 130 [19200/50000 (38%)]	Loss: 0.199977
Train Epoch: 130 [25600/50000 (51%)]	Loss: 0.103306
Train Epoch: 130 [32000/50000 (64%)]	Loss: 0.152193
Train Epoch: 130 [38400/50000 (77%)]	Loss: 0.167433
Train Epoch: 130 [44800/50000 (90%)]	Loss: 0.129756
测试集上准确率为: 53.72 %
Train Epoch: 131 [6400/50000 (13%)]	Loss: 0.056430
Train Epoch: 131 [12800/50000 (26%)]	Loss: 0.242675
Train Epoch: 131 [19200/50000 (38%)]	Loss: 0.578580
Train Epoch: 131 [25600/50000 (51%)]	Loss: 0.469197
Train Epoch: 131 [32000/50000 (64%)]	Loss: 0.507799
Train Epoch: 131 [38400/50000 (77%)]	Loss: 0.413032
Train Epoch: 131 [44800/50000 (90%)]	Loss: 0.297381
测试集上准确率为: 52.349999999999994 %
Train Epoch: 132 [6400/50000 (13%)]	Loss: 0.189013
Train Epoch: 132 [12800/50000 (26%)]	Lo

测试集上准确率为: 53.23 %
Train Epoch: 151 [6400/50000 (13%)]	Loss: 0.045486
Train Epoch: 151 [12800/50000 (26%)]	Loss: 0.106330
Train Epoch: 151 [19200/50000 (38%)]	Loss: 0.022998
Train Epoch: 151 [25600/50000 (51%)]	Loss: 0.079431
Train Epoch: 151 [32000/50000 (64%)]	Loss: 0.104377
Train Epoch: 151 [38400/50000 (77%)]	Loss: 0.160253
Train Epoch: 151 [44800/50000 (90%)]	Loss: 0.049530
测试集上准确率为: 53.14 %
Train Epoch: 152 [6400/50000 (13%)]	Loss: 0.109410
Train Epoch: 152 [12800/50000 (26%)]	Loss: 0.122342
Train Epoch: 152 [19200/50000 (38%)]	Loss: 0.010064
Train Epoch: 152 [25600/50000 (51%)]	Loss: 0.013006
Train Epoch: 152 [32000/50000 (64%)]	Loss: 0.087376
Train Epoch: 152 [38400/50000 (77%)]	Loss: 0.077892
Train Epoch: 152 [44800/50000 (90%)]	Loss: 0.125208
测试集上准确率为: 52.769999999999996 %
Train Epoch: 153 [6400/50000 (13%)]	Loss: 0.085070
Train Epoch: 153 [12800/50000 (26%)]	Loss: 0.089209
Train Epoch: 153 [19200/50000 (38%)]	Loss: 0.089059
Train Epoch: 153 [25600/50000 (51%)]	Loss: 0.054036


Train Epoch: 172 [19200/50000 (38%)]	Loss: 0.100469
Train Epoch: 172 [25600/50000 (51%)]	Loss: 0.120442
Train Epoch: 172 [32000/50000 (64%)]	Loss: 0.161474
Train Epoch: 172 [38400/50000 (77%)]	Loss: 0.066400
Train Epoch: 172 [44800/50000 (90%)]	Loss: 0.111020
测试集上准确率为: 52.52 %
Train Epoch: 173 [6400/50000 (13%)]	Loss: 0.063919
Train Epoch: 173 [12800/50000 (26%)]	Loss: 0.124619
Train Epoch: 173 [19200/50000 (38%)]	Loss: 0.199451
Train Epoch: 173 [25600/50000 (51%)]	Loss: 0.047037
Train Epoch: 173 [32000/50000 (64%)]	Loss: 0.041123
Train Epoch: 173 [38400/50000 (77%)]	Loss: 0.023290
Train Epoch: 173 [44800/50000 (90%)]	Loss: 0.029561
测试集上准确率为: 52.5 %
Train Epoch: 174 [6400/50000 (13%)]	Loss: 0.249840
Train Epoch: 174 [12800/50000 (26%)]	Loss: 0.067129
Train Epoch: 174 [19200/50000 (38%)]	Loss: 0.099961
Train Epoch: 174 [25600/50000 (51%)]	Loss: 0.130312
Train Epoch: 174 [32000/50000 (64%)]	Loss: 0.042876
Train Epoch: 174 [38400/50000 (77%)]	Loss: 0.152272
Train Epoch: 174 [44800/50000 (

Train Epoch: 193 [32000/50000 (64%)]	Loss: 0.172490
Train Epoch: 193 [38400/50000 (77%)]	Loss: 0.107200
Train Epoch: 193 [44800/50000 (90%)]	Loss: 0.064392
测试集上准确率为: 53.080000000000005 %
Train Epoch: 194 [6400/50000 (13%)]	Loss: 0.113554
Train Epoch: 194 [12800/50000 (26%)]	Loss: 0.060443
Train Epoch: 194 [19200/50000 (38%)]	Loss: 0.118391
Train Epoch: 194 [25600/50000 (51%)]	Loss: 0.074223
Train Epoch: 194 [32000/50000 (64%)]	Loss: 0.038135
Train Epoch: 194 [38400/50000 (77%)]	Loss: 0.119069
Train Epoch: 194 [44800/50000 (90%)]	Loss: 0.100854
测试集上准确率为: 53.580000000000005 %
Train Epoch: 195 [6400/50000 (13%)]	Loss: 0.021361
Train Epoch: 195 [12800/50000 (26%)]	Loss: 0.019507
Train Epoch: 195 [19200/50000 (38%)]	Loss: 0.136998
Train Epoch: 195 [25600/50000 (51%)]	Loss: 0.072245
Train Epoch: 195 [32000/50000 (64%)]	Loss: 0.082802
Train Epoch: 195 [38400/50000 (77%)]	Loss: 0.057213
Train Epoch: 195 [44800/50000 (90%)]	Loss: 0.088449
测试集上准确率为: 53.480000000000004 %
Train Epoch: 196 [6400/50